In [1]:
input_file = "day10/input"
lines = [l.strip() for l in open(input_file, "r").readlines()]

In [2]:
import collections
import enum

class Direction(enum.Enum):
    NORTH = 1
    EAST = 2
    SOUTH = 3
    WEST = 4

class Pos(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def move(self, direction):
        match direction:
            case Direction.NORTH:
                return Pos(self.x, self.y - 1)
            case Direction.EAST:
                return Pos(self.x + 1, self.y)
            case Direction.SOUTH:
                return Pos(self.x, self.y + 1)
            case Direction.WEST:
                return Pos(self.x - 1, self.y)
    def __eq__(self, other):
        return type(self) == type(other) and self.x == other.x and self.y == other.y
    def __repr__(self):
        return f"Pos({self.x}, {self.y})"

def peek(tiles, pos):
    if pos.x < 0 or pos.x >= len(tiles[0]) or pos.y < 0 or pos.y >= len(tiles):
        return None
    return tiles[pos.y][pos.x]


tiles = lines
def next_step(tiles, pos, direction):
    current = peek(tiles, pos)
    match current:
        case 'S':
            # init direction
            if peek(tiles, pos.move(Direction.NORTH)) in "F7|":
                direction = Direction.NORTH
            elif peek(tiles, pos.move(Direction.EAST)) in "J7-":
                direction = Direction.EAST
            elif peek(tiles, pos.move(Direction.SOUTH)) in "JL|":
                direction = Direction.SOUTH
            elif peek(tiles, pos.move(Direction.WEST)) in "LF-":
                direction = Direction.WEST
            
            return direction
        case '-':
            return direction
        case '|':
            return direction
        case '7':
            match direction:
                case Direction.EAST:
                    return Direction.SOUTH
                case Direction.NORTH:
                    return Direction.WEST
        case 'J':
            match direction:
                case Direction.EAST:
                    return Direction.NORTH
                case Direction.SOUTH:
                    return Direction.WEST
        case 'F':
            match direction:
                case Direction.WEST:
                    return Direction.SOUTH
                case Direction.NORTH:
                    return Direction.EAST
        case 'L':
            match direction:
                case Direction.WEST:
                    return Direction.NORTH
                case Direction.SOUTH:
                    return Direction.EAST
    print(f"Unknown action for pos[{pos}] = {current}, direction = {direction}")


def get_steps(tiles):
    start = [Pos(l.index('S'), y) for l, y in zip(tiles, range(len(tiles))) if 'S' in l][0]

    direction = next_step(tiles, start, None)
    steps = [direction]
    pos = start.move(direction)
    while peek(tiles, pos) != 'S':
        direction = next_step(tiles, pos, direction)
        steps.append(direction)
        pos = pos.move(direction)
    return start, steps

start, loop = get_steps(tiles)
print(f"Part 1: {len(loop) // 2}")

Part 1: 6831


In [3]:
loop_positions = []
pos = start
for d in loop:
    pos = pos.move(d)
    loop_positions.append(pos)

# resolves S - substitutes it for the tile it stands for in place
def resolve_start(tiles, start_pos):
    n = start_pos.move(Direction.NORTH) in loop_positions and peek(tiles, start_pos.move(Direction.NORTH)) in "7|F"
    s = start_pos.move(Direction.SOUTH) in loop_positions and peek(tiles, start_pos.move(Direction.SOUTH)) in "|LJ"
    e = start_pos.move(Direction.EAST) in loop_positions and peek(tiles, start_pos.move(Direction.EAST)) in "-7J"
    w = start_pos.move(Direction.WEST) in loop_positions and peek(tiles, start_pos.move(Direction.WEST)) in "-LF"
    if len([x for x in [n,s,e,w] if x]) != 2:
        raise f"Expected exactly 2 connections for S"
    elif n and s: c = '|'
    elif n and e: c = 'L'
    elif n and w: c = 'J'
    elif s and e: c = 'F'
    elif s and w: c = '7'
    elif e and w: c = '-'
    else: raise f"Unexpected combination of connections for S"
    tiles[start_pos.y] = tiles[start_pos.y].replace('S', c)

resolve_start(tiles, start)
    
# Counting cells enclosed inside loop. 
# The idea is that a line originating in an inside cell will intersect the loop odd number of times.
# Line direction is arbitrary, I chose (-1, 0).
# The algorithm is counting loop's '|'. 'F---J' and 'L---7' work as a '|' too. ('F7' and 'LJ' don't)

def skip_dash(line, x):
    while x < len(line) and line[x] == '-':
        x += 1
    return x

n_inside = 0
for line, y in zip(tiles, range(len(tiles))):
    inside = False
    x = 0
    while x < len(line):
        c = line[x]
        if Pos(x, y) in loop_positions:
            match line[x]:
                case '|': inside = not inside
                case 'L':
                    x = skip_dash(line, x + 1)
                    match line[x]:
                        case '7': inside = not inside
                        case 'J': pass
                        case unexpected: raise f"Unexpected tile {unexpected}"
                case 'F':
                    x = skip_dash(line, x + 1)
                    match line[x]:
                        case 'J': inside = not inside
                        case '7': pass
                        case unexpected: raise f"Unexpected tile {unexpected}"
                case unexpected: raise f"Unexpected tile {unexpected}"
        else:
            if inside:
                n_inside += 1
        x += 1
print(f"Part 2: {n_inside}")

Part 2: 305
